In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# %% load packages
import locale
import sys
import os
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.getcwd()

'c:\\Users\\amaguaya\\OneDrive - Kienzle Automotive GmbH\\Desktop\\DLiE\\repo\\DLiE_forecast_13_06_25\\srs\\notebooks'

In [5]:
import sys
sys.path.append('C:/Users/amaguaya/OneDrive - Kienzle Automotive GmbH/Desktop/DLiE/repo/DLiE_forecast_13_06_25')

import os
print("Working directory:", os.getcwd())



Working directory: c:\Users\amaguaya\OneDrive - Kienzle Automotive GmbH\Desktop\DLiE\repo\DLiE_forecast_13_06_25\srs\notebooks


In [6]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, SimpleMLP, train_mlp, \
  prepare_train_test_tensors, build_mlp_rolling_forecasts, tune_mlp_hyperparameters, \
  DST_trafo, prepare_dataset_tensor_modified

from srs.utils.our_utils import run_forecast_step, run_forecast_step_modified
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset



### models_24h fitting for all no1-no5 regions separately.

In [ ]:
'''
  eval/test periods
  2023 - 365 days
  2024 - 366 days
'''
# rmse  # -- 31.68, rmse: 8.79, 8.798 with rolling

'\n  eval/test periods\n  2023 - 365 days\n  2024 - 366 days\n'

In [9]:
# set the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **************************************
# define dates for training and evaluation 
# **************************************
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE = '2023-01-01'
FINAL_DATE_EXPERIMENTS = '2024-12-31'
n_days_test = (pd.to_datetime(FINAL_DATE_EXPERIMENTS) - pd.to_datetime(INIT_TEST_DATE)).days + (1) # additional adjustment

repo_root = Path.cwd().parents[1]
mapcodes = ["NO1"] # ["NO1","NO2","NO3","NO4","NO5"]
maps_dict = {}

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df = pd.read_csv(csv_path, parse_dates=["time_utc"])
    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz="CET",
        seed=42,
        test_days= n_days_test , # Here it should be 1*365 for 1 year of test, and so on
        dtype=torch.float64,
    )

    # fix potential problems with dates after change time zone..
    train_dates_series       = pd.DatetimeIndex(sorted(train_dates))
    id_init_exp = train_dates_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_test_period = train_dates_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_exp = train_dates_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t = data_t[id_init_exp:(id_end_exp+1), :,:]
    train_dates = pd.Series(train_dates[id_init_exp:(id_end_exp+1)])

    print(f"days for test data ..{data_t.shape[0] - id_init_test_period}")
    
    maps_dict[code] = {
        "df": df,
        "data_t": data_t,
        "train_t": train_t,
        "train_dates": train_dates,
        "price_t": price_t
    }
maps_dict.keys()

lgbm24_by_zone   = {}
rmse_by_zone  = {}



for z in mapcodes:
    print(f"\n--- {z} ---")
    price_S      = maps_dict[z]["price_t"]
    data_array   = maps_dict[z]["data_t"]
    full_dates      = maps_dict[z]["train_dates"] # <-- here i change a bit the previuous function, prepare_data_tensor, it has all dates
    feature_names= maps_dict[z]["df"].columns[1:]
    full_date_series = pd.DatetimeIndex(sorted(full_dates))

    # evaluation days (all of 2024)
    train_start_idx = full_date_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_eval = full_date_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_eval = full_date_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    eval_start_idx = id_init_eval
    eval_end_idx  = id_end_eval
    N_s            = (eval_end_idx - eval_start_idx) + 1 # additional adjustment
    full_dates = pd.to_datetime(full_dates)

    

    # new features: WD - dummy for week days, price lags for Mon, Tue and Fri, day-ahead load lag
    WD             = [1,6,7]     
    PRICE_S_LAGS   = [1,2,7]
    DA_LAG         = [0]
    S              = 24
    #D             = 730

    # tensors to collect forecasts for THIS zone
    forecasts_zone = torch.full((N_s, S, 1), float("nan"),
                                dtype=torch.float64, device=device)

    # thread pool
    with ThreadPoolExecutor(max(1, os.cpu_count() // 2) + 1 ) as executor:
        futures = [
            executor.submit(
                run_forecast_step_modified,
                n,
                price_S,
                data_array,
                train_start_idx = train_start_idx,
                train_end_idx   = id_init_eval - 1,
                full_dates      = full_dates,
                wd              = WD,                
                price_s_lags    = PRICE_S_LAGS,
                da_lag          = DA_LAG,
                feature_names   = feature_names,   # reg_names
            )
            for n in range(N_s)
        ]

        for fut in as_completed(futures):
            try:
                n, lgbm24 = fut.result()
                forecasts_zone[n, :, 0] = lgbm24.detach().clone().to(forecasts_zone.dtype).to(forecasts_zone.device)
            except Exception as e:
                import traceback
                print(f"Thread crashed: {e}")
                traceback.print_exc()

                
    #   shape: (N_s, S)
    true_vals = price_S[eval_start_idx : eval_end_idx + 1].to(device)  
    
    # compute RMSE
    diff = forecasts_zone[:, :, 0] - true_vals
    rmse = torch.sqrt((diff**2).mean()).item()
    
    print(f"Zone {z} LBGM-24h RMSE: {rmse:.4f}")

    lgbm24_by_zone[z] = forecasts_zone[:, :, 0].cpu()
    rmse_by_zone[z]  = rmse

days for test data ..731

--- NO1 ---
Loop   0: train 2019-01-01 00:00:00 -> 2022-12-31 00:00:00, forecast 2023-01-01 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   1: train 2019-01-02 00:00:00 -> 2023-01-01 00:00:00, forecast 2023-01-02 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   2: train 2019-01-03 00:00:00 -> 2023-01-02 00:00:00, forecast 2023-01-03 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   3: train 2019-01-04 00:00:00 -> 2023-01-03 00:00:00, forecast 2023-01-04 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   4: train 2019-01-05 00:00:00 -> 2023-01-04 00:00:00, forecast 2023-01-05 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   5: train 2019-01-06 00:00:00 -> 2023-01-05 00:00:00, forecast 2023-01-06 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Lo

C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311

-> finished loop 2
Loop   7: train 2019-01-08 00:00:00 -> 2023-01-07 00:00:00, forecast 2023-01-08 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
-> finished loop 4
Loop   8: train 2019-01-09 00:00:00 -> 2023-01-08 00:00:00, forecast 2023-01-09 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
-> finished loop 0
Loop   9: train 2019-01-10 00:00:00 -> 2023-01-09 00:00:00, forecast 2023-01-10 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
-> finished loop 1
Loop  10: train 2019-01-11 00:00:00 -> 2023-01-10 00:00:00, forecast 2023-01-11 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


-> finished loop 3
Loop  11: train 2019-01-12 00:00:00 -> 2023-01-11 00:00:00, forecast 2023-01-12 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
-> finished loop 5
Loop  12: train 2019-01-13 00:00:00 -> 2023-01-12 00:00:00, forecast 2023-01-13 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
-> finished loop 6
Loop  13: train 2019-01-14 00:00:00 -> 2023-01-13 00:00:00, forecast 2023-01-14 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088


...

C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


-> finished loop 724
-> finished loop 726


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


-> finished loop 727
-> finished loop 728
-> finished loop 729
-> finished loop 730
Zone NO1 LBGM-24h RMSE: 8.7986


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
